In [1]:
%run ../../modulos.ipynb

# Carga a la base de datos

Consultas:

- Analizar el número de ingrados y exgrados de cada uno de los nodos (jugadores) y verificar el número de ingrados por equipo.
- Determiar qué jugadores tuvieron un torneo más difícil. Esto es, asignar a cada una de las aristas el cociente:

$$ \text{Valor de resultado} = \frac{\text{elo}_{loser}}{\text{elo}_{winner}} $$

Si el valor de resultado es mayor a 1, entonces tiene mucho mérito por parte del jugador más novato. Si el valor de resultado es menor a 1 y cercano a cero, entonces es un resultado esperado.

Si el promedio de ingrados es cercano a cero, te han ganado jugadores muy buenos y, por tanto, tuviste un torneo con el resultado esperado. Si el promedio de ingrados es mayor que 1, entonces te han ganado jugadores más débiles y, por tanto, tuviste mala suerte.

Si el promedio de exgrados es mayor a 1, tuviste un torneo muy bueno pues derrotaste a gente muy buena. Si el promedio de los exgrados es menor a 1, entonces ganaste sobre jugadores más débiles.

- Jugadores sorpresa con un valor de resultado promedio mayor a 1.

In [2]:
import ndjson

In [3]:
def obtener_data_lichess(url):    
    headers = {"Authorization": f"Bearer {personal_access_token}"}    
    response = requests.get(url, headers=headers)    
    return response

In [14]:
juegos_universitarios = obtener_data_lichess('https://lichess.org/api/tournament/jszDSiUx/games')

Cada línea del conjunto de datos puede verse como el item de una lista mediante la siguiente instrucción.

In [15]:
games = [s for s in juegos_universitarios.text.split("\n")[:-1]]

Cada línea puede ser agregada a Mongo, pero el objetivo es agregar los datos a Neo4j.

In [26]:
from neo4j import GraphDatabase

data_base_connection = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "1234"))
session = data_base_connection.session()

In [27]:
# elimina toda la base de datos
session.run('MATCH (n) DETACH DELETE n')
q=session.run('MATCH (n) RETURN n')

In [20]:
def agregar_neo4j(white, white_rating, white_team, black, black_rating, black_team, result, valor_resultado):
    instruccion = f'''
    MERGE (u1:Usuario {{id:"{white}"}})
    MERGE (u2:Usuario {{id:"{black}"}})
    MERGE (t1:Equipo {{nombre_equipo:"{white_team}"}})
    MERGE (t2:Equipo {{nombre_equipo:"{black_team}"}})
    MERGE (u1)-[:juega_para]->(t1)
    MERGE (u2)-[:juega_para]->(t2)
    '''

    if result == 1:
        instruccion += f'MERGE (u1)-[g:gano_a {{valor_resultado:"{valor_resultado}", white_rating:"{white_rating}", black_rating:"{black_rating}"}}]->(u2)'
    elif result == 0:
        instruccion += f'MERGE (u2)-[g:gano_a {{valor_resultado:"{valor_resultado}", white_rating:"{white_rating}", black_rating:"{black_rating}"}}]->(u1)'
    elif result == 0.5:
        if white_rating >= black_rating:
            instruccion += f'MERGE (u2)-[g:gano_a {{valor_resultado:"{valor_resultado}", white_rating:"{white_rating}", black_rating:"{black_rating}"}}]->(u1)'
        else:
            instruccion += f'MERGE (u1)-[g:gano_a {{valor_resultado:"{valor_resultado}", white_rating:"{white_rating}", black_rating:"{black_rating}"}}]->(u2)'
    
    return instruccion

In [28]:
flag_valor_resultado = False
jugadores = set()
equipos = set()
result = 1000

# iteramos sobre cada una de las líneas de games
for i in tqm(games[:]):

    # extracción de usuarios
    if i.count('White "'):
        white_user_id = i[8:-2]
        jugadores.add(white_user_id)
    if i.count('Black "'):
        black_user_id = i[8:-2]
        jugadores.add(black_user_id)

    # extracción de resultado
    if i.count('Result "1-0"'):
        result = 1
    elif i.count('Result "0-1"'):
        result = 0
    elif i.count('Result "1/2-1/2"'):
        result = 0.5

    # extracción de rating
    if i.count('WhiteElo'):
        white_rating = float(i[11:-2])
    if i.count('BlackElo'):
        black_rating = float(i[11:-2])
        flag_valor_resultado = True

    # extracción de equipo
    if i.count('WhiteTeam'):
        white_team = i[12:-2]
        equipos.add(white_team)
    if i.count('BlackTeam'):
        black_team = i[12:-2]
        equipos.add(black_team)
        # agregar datos a Neo4j
        instruccion = agregar_neo4j(white_user_id,
                                    white_rating,
                                    white_team,
                                    black_user_id,
                                    black_rating,
                                    black_team,
                                    result,
                                    valor_de_resultado)
        session.run(instruccion)

    # cálculo de valor de resultado
    if (result == 1) & (flag_valor_resultado == True):
        valor_de_resultado = np.round(black_rating / white_rating, 2)
        flag_valor_resultado = False
    elif (result == 0) & (flag_valor_resultado == True):
        valor_de_resultado = np.round(white_rating / black_rating, 2)
        flag_valor_resultado = False
    elif (result == 0.5) & (flag_valor_resultado == True):
        if white_rating >= black_rating:
            # deberá tener penalización el jugador con más rating que entabló
            valor_de_resultado = np.round(white_rating / black_rating, 2)
            flag_valor_resultado = False
        else:
            valor_de_resultado = np.round(black_rating / white_rating, 2)
            flag_valor_resultado = False

  0%|          | 0/22021 [00:00<?, ?it/s]

In [5]:
def exgrados_jugador(jugador):
    """
    Obtiene los puntos por jugador via los exgrados de su nodo.
    """

    instruccion = f'''
    MATCH (u1:Usuario)
    MATCH (u2:Usuario)
    WHERE u1.id = '{jugador}'
    MATCH (u1)-[:gano_a]->(u2)
    RETURN count(*) as exgrados
    '''
    return session.run(instruccion).data()[0]['exgrados']

def equipo_jugador(jugador):
    """
    Obtiene el equipo de un jugador dado desde la base de Neo4j.
    """

    instruccion = f'''
    MATCH (u:Usuario)
    MATCH (t:Equipo)
    WHERE u.id = '{jugador}'
    MATCH (u)-[:juega_para]->(t)
    return t
    '''

    return session.run(instruccion).data()[0]['t']['nombre_equipo']

def victorias_sobresalientes(jugador):
    """
    Cuenta el número de partidas ganadas contra jugadores con
    un rating mayor.
    """

    instruccion = f'''
    MATCH (u1:Usuario)
    MATCH (u2:Usuario)
    WHERE u1.id = '{jugador}'
    MATCH (u1)-[g:gano_a]->(u2)
    RETURN g.valor_resultado
    '''

    count = 0

    for resultado in session.run(instruccion).data():
        if float(resultado['g.valor_resultado']) > 1:
            count += 1

    return count

def derrotas_jugador(jugador):
    """
    Número de derrotas por jugador.
    """

    instruccion = f'''
    MATCH (u1:Usuario)
    MATCH (u2:Usuario)
    WHERE u1.id = '{jugador}'
    MATCH (u2)-[g:gano_a]->(u1)
    RETURN count(*) as ingrados
    '''

    return session.run(instruccion).data()[0]['ingrados']

def peores_derrotas(jugador):
    """
    Cuenta el número de derrotas contra jugadores inferiores
    en el momento de la partida.
    """

    instruccion = f'''
    MATCH (u1:Usuario)
    MATCH (u2:Usuario)
    WHERE u1.id = '{jugador}'
    MATCH (u2)-[g:gano_a]->(u1)
    RETURN g.valor_resultado
    '''

    count = 0

    for resultado in session.run(instruccion).data():
        if float(resultado['g.valor_resultado']) > 1:
            count += 1

    return count

# Ranking por equipos

Con el número de exgrados por jugador de cada equipo podemos determinar qué equipo hizo más puntos.

In [29]:
# diccionario para almacenar resultados
ranking_equipos = {}
for equipo in equipos:
    ranking_equipos[equipo] = 0

# exgrados por participante
for jugador in tqm(jugadores):
    puntos = exgrados_jugador(jugador)
    equipo = equipo_jugador(jugador)
    ranking_equipos[equipo] += puntos

  0%|          | 0/115 [00:00<?, ?it/s]

In [49]:
# DataFrame para realizar gráfico
ranking_equipos_df = pd.DataFrame({'Institución':ranking_equipos.keys(),
                                   'Puntos':ranking_equipos.values()})

# gráfica
fig = px.bar(ranking_equipos_df.sort_values('Puntos', ascending=False), x='Institución', y='Puntos', color='Institución', opacity=0.6)
fig.update_layout(
    showlegend=False,
    template='plotly_dark'
    )
fig.show()

![img1](./imgs/rank.png)


# Posibles jugadores tramposos o que sobresalieron

In [32]:
# diccionario para almacenar resultados
resultados_sobresalientes = {}

# exgrados por participante
for jugador in tqm(jugadores):
    resultados_sobresalientes[jugador] = victorias_sobresalientes(jugador)

  0%|          | 0/115 [00:00<?, ?it/s]

In [52]:
# DataFrame para realizar gráfico
resultados_sobresalientes_df = pd.DataFrame({'Jugador':resultados_sobresalientes.keys(),
                                   'Puntos destacados':resultados_sobresalientes.values()})

# gráfica
fig = px.bar(resultados_sobresalientes_df.sort_values('Puntos destacados', ascending=False).iloc[:10, :], x='Jugador', y='Puntos destacados', color='Jugador', opacity=0.6)
fig.update_layout(
    showlegend=False,
    template='plotly_dark'
    )
fig.show()

![img2](./imgs/unknown.png)

# Jugadores con torneo más complicado

In [42]:
# diccionario para almacenar resultados
resultados_malos = {}

# ingrados por participante
for jugador in tqm(jugadores):
    resultados_malos[jugador] = derrotas_jugador(jugador)

  0%|          | 0/115 [00:00<?, ?it/s]

In [51]:
# DataFrame para realizar gráfico
resultados_malos_df = pd.DataFrame({'Jugador':resultados_malos.keys(),
                                   'Ingrados':resultados_malos.values()})

# gráfica
fig = px.bar(resultados_malos_df.sort_values('Ingrados', ascending=False).iloc[:10, :], x='Jugador', y='Ingrados', color='Jugador', opacity=0.6)
fig.update_layout(
    showlegend=False,
    template='plotly_dark'
    )
fig.show()

![img3](./imgs/unknown1)

# Jugadores que más perdieron contra oponentes inferiores

In [44]:
# diccionario para almacenar resultados
resultados_inferiores = {}

# ingrados por participante
for jugador in tqm(jugadores):
    resultados_inferiores[jugador] = peores_derrotas(jugador)

  0%|          | 0/115 [00:00<?, ?it/s]

In [50]:
# DataFrame para realizar gráfico
peores_derrotas_df = pd.DataFrame({'Jugador':resultados_inferiores.keys(),
                                   'Cuenta':resultados_inferiores.values()})

# gráfica
fig = px.bar(peores_derrotas_df.sort_values('Cuenta', ascending=False).iloc[:10, :], x='Jugador', y='Cuenta', color='Jugador', opacity=0.6)
fig.update_layout(
    showlegend=False,
    template='plotly_dark'
    )
fig.show()

![img4](./imgs/unknown2.png)